In [228]:

import re

In [229]:
def get_lines_from_uniprot(code):
    import requests
    url = f'https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query={code}'
    all_fastas = requests.get(url).text
    lines = all_fastas.split('\n')
    return lines

In [230]:
def get_string_from_lines(lines, id2):
    if len(id2):
        header_idxs = [i for i, l in enumerate(lines) if len(l) and l[0] == '>' and id2 in l]
        if not len(header_idxs):
            print(id2, [l.split(' ')[0] for i, l in enumerate(lines) if len(l) and l[0] == '>'])
            return
        header_idx = min(header_idxs)
        string = ''
        for l in lines[header_idx + 1:]:
            if len(l) and l[0] != '>':
                string += l
            else:
                break
        return string
    else:
        return "".join(lines[1:])

In [231]:
def find_all_matches(pattern, string):
    pat = re.compile(pattern)
    pos = 0
    out = []
    positions = []
    while (match := pat.search(string, pos)) is not None:
        pos = match.start() + 1
        positions.append(pos)
        out.append(match[0])
    return out, positions

In [232]:
example = [
"A1USX4",
"B6DCT5",
"A5WBR3",
"A2Z669",
"P22891_PROZ_HUMAN",
"B3CNE0",
"Q6GEK4",
"P04441_HG2A_MOUSE",
"A5GIU0",
"P19835_CEL_HUMAN",
"P08318_P150_HCMVA",
"O13188",
"P01787",
"P01045_KNG2_BOVIN",

]
for id in example:
    id1 = id[:6]
    id2 = id[7:]
    lines = get_lines_from_uniprot(id1)
    string = get_string_from_lines(lines, id2)
    matches, positions = find_all_matches('N[^P][ST][^P]', string)
    if len(positions):
        print(id)
        pos_str = ''
        for p in positions:
            pos_str += str(p) + ' '
        print(pos_str)

A5WBR3
5 
P22891_PROZ_HUMAN
99 225 233 306 332 
B3CNE0
107 
P04441_HG2A_MOUSE
113 119 239 253 
P19835_CEL_HUMAN
207 
P08318_P150_HCMVA
833 
O13188
207 
P01045_KNG2_BOVIN
47 87 168 169 197 204 280 
